This file is used to calculate the roots of unity from which the twiddle factors are derived.

In [ ]:
from sympy.ntheory.residue_ntheory import nthroot_mod

small_prime = 7681
solinas_prime = int(0xFFFFFFFF00000001)


In [ ]:
# basic functions

def gcdExtended(a, b): 
    # source: https://www.geeksforgeeks.org/python-program-for-basic-and-extended-euclidean-algorithms-2/
    # Base Case 
    if a == 0 : 
        return b,0,1             
    gcd,x1,y1 = gcdExtended(b%a, a)      
    # Update x and y using results of recursive call 
    x = y1 - (b//a) * x1 
    y = x1     
    return gcd,x,y

def get_params(polym_size, prime):
    w_list = nthroot_mod(1, polym_size, prime, all_roots=True)
    w = 0
    for num in w_list:
        # check w^k != 1 mod q with k < n
        ok = True
        for k in range(1, polym_size):
            if x_to_pow_y_mod_z_scalar(num, k, prime) == 1:
                ok = False
                break
        if ok:
            w = num
            break
    if w == 0:
        raise Exception("Could not find matching w")

    w_2n_list = nthroot_mod(w, 2, prime, all_roots=True)
    w_2n = 0
    for num in w_2n_list:        
        if x_to_pow_y_mod_z_scalar(num, polym_size, prime) == -1 or x_to_pow_y_mod_z_scalar(num, polym_size, prime) == prime-1:
            w_2n = num
            break
    if w_2n == 0:
        raise Exception("Could not find matching w_2n")
    
    w_invers = gcdExtended(w, prime)[1]
    if w_invers < 0:
        w_invers = w_invers + prime
    
    n_invers = gcdExtended(polym_size, prime)[1]
    if n_invers < 0:
        n_invers = n_invers + prime
    
    w_2n_invers = gcdExtended(w_2n, prime)[1]
    if w_2n_invers < 0:
        w_2n_invers = w_2n_invers + prime
    
    return polym_size, prime, w, w_invers, w_2n, w_2n_invers, n_invers

def x_to_pow_y_mod_z_scalar(x:int,y:int,z:int):
    # we use this to avoid overflows
    temp:int = 1
    if y < 0:
        raise("Sorry - not implemented for negative exponents!")
    else:
        for i in range(y):
            temp = (temp*x) % z
    return temp

def print_hexadecimal(int_array):
    for my_int in int_array:
        print("0x{0:0x}".format(int(my_int)))


In [ ]:
# compute prime-related parameters for different polynomial sizes
test_prime = solinas_prime
max_power_of_2 = 10
for i in range(max_power_of_2):
    try:
        params = get_params(2**i, test_prime)
        relevant_params = [params[6], params[2], params[3]] # n_invers, w, w_invers
        print("Integer form: ",relevant_params, "In hexadecimal:")
        for idx, param in enumerate(relevant_params):
            print("0x{0:0x}".format(param))
    except:
        print(f"No params found for {i}")